In [ ]:
import boto3 
import os
import json
from dotenv import load_dotenv
from botocore.exceptions import ClientError


load_dotenv()

aws_secret_key = os.environ["AWS_SECRET_ACCESS_KEY"]
aws_access_key_id = os.environ["AWS_ACCESS_KEY_ID"]
aws_region = os.environ["AWS_REGION"]

In [ ]:
print(aws_access_key_id)
print(aws_secret_key)
print(aws_region)

In [ ]:
client = boto3.client("bedrock-runtime", region_name="us-west-2", aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_key)


In [ ]:
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

In [1]:
SYSTEM_PROMPT = """
This is dictated by a doctor to a voice assistant to generate a clinical note based on this information. While dictating, there might be transcription errors (e.g., wrong medical terms, drug names, or procedures).
          Your task:
          1. Identify and correct any transcription mistakes in the dictation (for example, "iron olecranon" → "isotretinoin", "creatinine" → "tretinoin", "resistantectomy" → "resistant acne").
          2. Extract the clinical information and create a structured clinical note in the following JSON format:
          {
            "past_medical_history": "",
            "allergies": "",
            "current_medication": "",
            "review_of_system": "",
            "history_of_present_illness": "",
            "examination": "",
            "assessment_and_plan": "",
            "procedure": "",
            "icdCodes": [{
              "Code": "",
              "Description": ""
            }],
            "cptCodes": [{
              "Code": "",
              "Description": ""
            }]
          }
          Rules:
          - Only include information that is explicitly stated or clearly implied in the transcript.
          - Correct transcription errors intelligently using medical context.
          - Give Correct icdCodes and cptCodes for the clinical note
          - Must Produce HTML for each sections. Use proper heading for diagnosis names bold and underlined and don't use bullet points or numbering and put them in the JSON format provided above.
          - You may find some HTML templates in the transcript, You must keep the HTML templates unchanged in the transcript and put the necessary information in those structured templates.
          Strict Rule: Do not add any headings or section titles inside the content of the JSON fields. The keys in the JSON object (e.g., "past_medical_history", "history_of_present_illness", "assessment_and_plan") already act as section identifiers. The HTML content inside each field must only include the relevant information, not an extra heading or label.
          - If atleast one of the field contains relevant data, return the JSON object with all the fields (empty or filled).
          - If no relevant or sufficient medical content exists, return:
          {
            "error": "Insufficient or unrelated content"
          }
          - Format the note dynamically based on the requested style:
            - Focused → concise, essential details only.
            - Comprehensive → detailed, narrative-style documentation.
            - Categorized → structured, bullet-like content inside each field.
          - Output must be ONLY the JSON object
          
          """

In [10]:
import boto3
import json

def chat_with_claude(prompt, region='us-east-1'):
    """
    Chat with Claude via AWS Bedrock using inference profiles
    """
    bedrock = boto3.client('bedrock-runtime', region_name=region)
    
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 2000,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    })
    
    # Use cross-region inference profile for Claude 3.5 Sonnet v2
    model_id = 'us.anthropic.claude-sonnet-4-5-20250929-v1:0'
    
    try:
        response = bedrock.invoke_model(
            modelId=model_id,
            body=body
        )
        
        result = json.loads(response['body'].read())
        return result['content'][0]['text']
        
    except Exception as e:
        return f"Error: {str(e)}"   
# Test it
if __name__ == "__main__":
    response = chat_with_claude("Hello! Please introduce yourself briefly.")
    print(response)

Hello! I'm Claude, an AI assistant made by Anthropic. I'm here to help you with a wide variety of tasks like answering questions, writing, analysis, math, coding, and creative projects. I aim to be helpful, harmless, and honest in our conversations. What can I help you with today?
